# Using SVM to predict Grainsize

TL;DR, RBF is best kernal. Poly does not work.

Put together by Thomas Martin, thomasmartin@mines.edu, all errors are mine

In [1]:
# If you have installation questions, please reach out
import pandas as pd # data storage
import numpy as np  # math and stuff

import sklearn  

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import mean_squared_error

from sklearn.utils.class_weight import compute_sample_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score, max_error, mean_squared_error

from sklearn import svm
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt # plotting utility

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv('drive/My Drive/1_lewis_research/core_to_wl_merge/Merged_dataset_inner_imputed_12_21_2020.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df = df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'LiveTime2','ScanTime2', 'LiveTime1','ScanTime1',
              'ref_num', 'API', 'well_name', 'sample_num' ], axis=1)

print(df.columns.values)

df.describe()

['depth_ft' 'USGS_ID' 'CAL' 'GR' 'DT' 'SP' 'DENS' 'PE' 'RESD' 'PHIN'
 'PHID' 'PE_smooth' 'GR_smooth' 'gz_linear_interp' 'gz_pchip_interp'
 'perm_air_md' 'perm_klink_md' 'He_por' 'horz_perm_md' 'effective_por'
 'por_percent' 'stress_swanson_perm_md' 'sample' 'Na' 'Na Err' 'Mg'
 'Mg Err' 'Al' 'Al Err' 'Si' 'Si Err' 'P' 'P Err' 'S' 'S Err' 'K' 'K Err'
 'Ca' 'Ca Err' 'Ti' 'Ti Err' 'Cr' 'Cr Err' 'Mn' 'Mn Err' 'Fe' 'Fe Err'
 'Co' 'Co Err' 'Ni' 'Ni Err' 'Cu' 'Cu Err' 'Zn' 'Zn Err' 'Ba' 'Ba Err']


,depth_ft,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,PE_smooth,GR_smooth,gz_linear_interp,gz_pchip_interp,perm_air_md,perm_klink_md,He_por,horz_perm_md,effective_por,por_percent,stress_swanson_perm_md,Na,Na Err,Mg,Mg Err,Al,Al Err,Si,Si Err,P,P Err,S,S Err,K,K Err,Ca,Ca Err,Ti,Ti Err,Cr,Cr Err,Mn,Mn Err,Fe,Fe Err,Co,Co Err,Ni,Ni Err,Cu,Cu Err,Zn,Zn Err,Ba,Ba Err
count,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,12030.000000,508.000000,510.000000,510.000000,25.000000,25.000000,24.000000,24.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.0,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000
mean,5397.863358,0.468406,0.325999,0.430844,0.811259,0.613371,0.351642,1.011018,0.172737,0.172737,0.351702,0.326029,4.913576,4.910244,81.002187,73.797216,14.267039,8.304000,11.808000,8.420833,1.646585,1.132042,0.025387,0.762913,0.011739,3.490170,0.012272,32.981494,0.033185,0.048516,0.001821,0.728955,0.002247,1.675288,0.005710,2.779623,0.005543,0.190546,0.001373,0.005787,0.000116,0.016520,0.000705,1.941109,0.003196,0.001315,0.0,0.004773,0.000100,0.003211,0.000822,0.006309,0.000763,0.090289,0.004689
std,5106.156677,0.191316,0.237404,0.263514,0.162843,0.214604,0.186912,1.503608,0.226664,0.226664,0.178735,0.229901,1.738128,1.751499,173.917355,161.251984,9.365678,3.161814,4.446902,3.050728,3.741652,0.777851,0.004405,0.488501,0.002843,1.415185,0.002607,4.940991,0.002003,0.023636,0.000221,0.795110,0.001252,0.761818,0.001383,2.045850,0.002189,0.088087,0.000297,0.002270,0.000075,0.024982,0.000128,1.472163,0.001193,0.000821,0.0,0.000796,0.000002,0.004429,0.000114,0.004559,0.000100,0.065308,0.000594
min,150.000000,0.000000,0.000000,0.003752,0.000000,0.000000,0.000000,-0.744314,0.000570,0.000570,0.004477,0.000258,1.204296,1.199972,0.001000,0.000000,1.000000,1.900000,5.900000,4.000000,0.000390,0.126200,0.016000,0.047000,0.006800,0.992200,0.007100,8.429500,0.021700,0.000000,0.001100,0.040300,0.000600,0.118400,0.002000,0.133500,0.001400,0.048500,0.000800,0.000000,-0.001200,0.000000,0.000500,0.270700,0.001300,0.000200,0.0,0.002700,0.000100,0.000000,0.000200,0.000000,0.000500,0.000000,0.003100
25%,450.725000,0.360072,0.122318,0.174146,0.743878,0.424460,0.216157,-0.037336,0.004595,0.004595,0.220800,0.131058,3.538724,3.531571,0.040750,0.020250,8.300000,7.900000,8.400000,6.800000,0.001140,0.768500,0.023400,0.293850,0.009100,2.411025,0.010200,29.728050,0.032000,0.032400,0.001700,0.182225,0.001200,1.069725,0.004600,1.103025,0.003600,0.121875,0.001100,0.004400,0.000100,0.003200,0.000600,0.964250,0.002300,0.000700,0.0,0.004300,0.000100,0.000000,0.000700,0.002700,0.000700,0.053200,0.004300
50%,1614.650000,0.554827,0.275825,0.471657,0.867010,0.605516,0.329327,0.795008,0.005532,0.005532,0.337049,0.283192,4.307541,4.256191,0.065500,0.037500,9.300000,9.500000,10.600000,7.300000,0.002070,1.057300,0.025200,0.692000,0.011700,3.133250,0.011700,31.893100,0.033300,0.047250,0.001800,0.461450,0.001900,1.560450,0.005800,2.593250,0.005700,0.160300,0.001300,0.005400,0.000100,0.014200,0.000700,1.658800,0.003100,0.001200,0.0,0.004800,0.000100,0.000000,0.000800,0.005400,0.000800,0.079300,0.004700
75%,9649.575000,0.602566,0.530980,0.662807,0.925012,0.830179,0.470706,1.378387,0.332503,0.332503,0.463317,0.525448,6.615175,6.633493,71.375000,59.675000,26.610000,10.300000,13.800000,9.200000,0.949777,1.322750,0.026700,1.192400,0.014400,4.531475,0.014400,37.944075,0.034700,0.061800,0.002000,0.941100,0.002900,2.291400,0.006900,3.872700,0.007000,0.258025,0.001600,0.006800,0.000100,0.023400,0.000800,2.616775,0.003900,0.001700,0.0,0.005200,0.000100,0.006300,0.000900,0.009200,0.0

In [5]:
df = df[df['Si']>0]

In [6]:
#df = df[df.USGS_ID != 'E997']

In [7]:
df.describe()

,depth_ft,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,PE_smooth,GR_smooth,gz_linear_interp,gz_pchip_interp,perm_air_md,perm_klink_md,He_por,horz_perm_md,effective_por,por_percent,stress_swanson_perm_md,Na,Na Err,Mg,Mg Err,Al,Al Err,Si,Si Err,P,P Err,S,S Err,K,K Err,Ca,Ca Err,Ti,Ti Err,Cr,Cr Err,Mn,Mn Err,Fe,Fe Err,Co,Co Err,Ni,Ni Err,Cu,Cu Err,Zn,Zn Err,Ba,Ba Err
count,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,278.000000,278.000000,278.000000,25.000000,25.000000,24.000000,24.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.0,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000,2396.000000
mean,5381.097037,0.468367,0.325982,0.431237,0.811337,0.613171,0.351816,1.011043,0.172725,0.172725,0.351716,0.326086,4.929909,4.927045,10.063730,9.155464,9.206727,8.304000,11.808000,8.420833,1.646585,1.132042,0.025387,0.762913,0.011739,3.490170,0.012272,32.981494,0.033185,0.048516,0.001821,0.728955,0.002247,1.675288,0.005710,2.779623,0.005543,0.190546,0.001373,0.005787,0.000116,0.016520,0.000705,1.941109,0.003196,0.001315,0.0,0.004773,0.000100,0.003211,0.000822,0.006309,0.000763,0.090289,0.004689
std,5101.029774,0.191645,0.237586,0.263125,0.163012,0.214509,0.187277,1.499012,0.227024,0.227024,0.178678,0.229953,1.742970,1.756868,71.398097,66.537002,4.492513,3.161814,4.446902,3.050728,3.741652,0.777851,0.004405,0.488501,0.002843,1.415185,0.002607,4.940991,0.002003,0.023636,0.000221,0.795110,0.001252,0.761818,0.001383,2.045850,0.002189,0.088087,0.000297,0.002270,0.000075,0.024982,0.000128,1.472163,0.001193,0.000821,0.0,0.000796,0.000002,0.004429,0.000114,0.004559,0.000100,0.065308,0.000594
min,150.000000,0.000000,0.000000,0.003880,0.007154,0.001136,0.000098,-0.744314,0.000664,0.000664,0.004685,0.000377,1.238052,1.231032,0.001000,0.001000,1.000000,1.900000,5.900000,4.000000,0.000390,0.126200,0.016000,0.047000,0.006800,0.992200,0.007100,8.429500,0.021700,0.000000,0.001100,0.040300,0.000600,0.118400,0.002000,0.133500,0.001400,0.048500,0.000800,0.000000,-0.001200,0.000000,0.000500,0.270700,0.001300,0.000200,0.0,0.002700,0.000100,0.000000,0.000200,0.000000,0.000500,0.000000,0.003100
25%,450.375000,0.362761,0.121491,0.176235,0.743371,0.423086,0.216361,-0.033914,0.004595,0.004595,0.220855,0.131464,3.541975,3.534175,0.034000,0.016250,7.900000,7.900000,8.400000,6.800000,0.001140,0.768500,0.023400,0.293850,0.009100,2.411025,0.010200,29.728050,0.032000,0.032400,0.001700,0.182225,0.001200,1.069725,0.004600,1.103025,0.003600,0.121875,0.001100,0.004400,0.000100,0.003200,0.000600,0.964250,0.002300,0.000700,0.0,0.004300,0.000100,0.000000,0.000700,0.002700,0.000700,0.053200,0.004300
50%,1613.250000,0.554827,0.275724,0.472182,0.867582,0.605516,0.329464,0.801593,0.005520,0.005520,0.335836,0.282739,4.325345,4.295978,0.059000,0.033000,9.000000,9.500000,10.600000,7.300000,0.002070,1.057300,0.025200,0.692000,0.011700,3.133250,0.011700,31.893100,0.033300,0.047250,0.001800,0.461450,0.001900,1.560450,0.005800,2.593250,0.005700,0.160300,0.001300,0.005400,0.000100,0.014200,0.000700,1.658800,0.003100,0.001200,0.0,0.004800,0.000100,0.000000,0.000800,0.005400,0.000800,0.079300,0.004700
75%,9484.625000,0.602566,0.530504,0.662435,0.924986,0.830135,0.470435,1.378201,0.332579,0.332579,0.462545,0.525624,6.621108,6.636347,0.072750,0.041000,9.800000,10.300000,13.800000,9.200000,0.949777,1.322750,0.026700,1.192400,0.014400,4.531475,0.014400,37.944075,0.034700,0.061800,0.002000,0.941100,0.002900,2.291400,0.006900,3.872700,0.007000,0.258025,0.001600,0.006800,0.000100,0.023400,0.000800,2.616775,0.003900,0.001700,0.0,0.005200,0.000100,0.006300,0.000900,0.009200,0.000800,0.106600,0.0051

## Loading in Dataset

In [8]:
dataset = df[[
  'depth_ft', 'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
  'RESD', 'PHIN', 'PHID', 'GR_smooth', 'PE_smooth',
  'Si'
]]

In [9]:
dataset.head(3)

,depth_ft,CAL,GR,DT,SP,DENS,PE,RESD,PHIN,PHID,GR_smooth,PE_smooth,Si
0,7741.0,0.630495,0.187403,0.054025,0.582081,0.829176,0.349137,0.982069,0.001692,0.001692,0.183350,0.366294,33.9921
5,7741.5,0.626112,0.182393,0.063209,0.586778,0.825487,0.351442,0.922670,0.001810,0.001810,0.196369,0.357995,33.9915
10,7742.0,0.621729,0.184446,0.063426,0.590484,0.825878,0.402086,0.886733,0.001889,0.001889,0.211611,0.360458,33.3257


In [10]:
X = dataset[['depth_ft', 'CAL', 'GR', 'DT', 'SP', 'DENS', 'PE',
        'RESD', 'PHIN', 'PHID', 'GR_smooth', 'PE_smooth']]

Y = dataset[['Si']]

Y_array = np.array(Y.values)

## Starting to set up the ML model params

In [11]:
seed = 7 # random seed is only used if you want to compare exact answers with friends 
test_size = 0.25 # how much data you want to withold, .15 - 0.3 is a good starting point

X_train, X_test, y_train, y_test = train_test_split(X.values, Y_array, test_size=test_size)

In [12]:
svm_rbf = svm.SVR(kernel='rbf')

svm_rbf.fit(X_train, y_train)

preds = svm_rbf.predict(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [13]:
rmse2 = mean_squared_error(y_test, preds, squared=False)
print("Mean Squared Error: %f" % (rmse2))
max1 = np.sqrt(max_error(y_test, preds))
print("Max Error: %f" % (max1))

Mean Squared Error: 3.443655
Max Error: 3.866528


# RBF Kernal Hyperparameter Tune

In [20]:
parameters = {
    'C': [3, 4, 5, 6, 8],
    'tol':[0.001, 0.01, 0.2, 0.4],
    'gamma': ['auto', 'scale', 0.1, 1, 3, 10],
    'max_iter': [500, 1000],
    'epsilon': [0.4, 0.6, 0.8, 1.4, 1.6],
    'degree': [2, 3]
}

estimator = svm.SVR(kernel='rbf')

grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    n_jobs = 6,
    cv = 3,
    verbose = True
)

grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 2400 candidates, totalling 7200 fits


[Parallel(n_jobs=6)]: Using backend LokyBackend with 6 concurrent workers.
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    3.4s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   15.8s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:   35.7s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:  1.0min
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:  1.6min
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:  2.3min
[Parallel(n_jobs=6)]: Done 2438 tasks      | elapsed:  3.1min
[Parallel(n_jobs=6)]: Done 3188 tasks      | elapsed:  4.1min
[Parallel(n_jobs=6)]: Done 4038 tasks      | elapsed:  5.1min
[Parallel(n_jobs=6)]: Done 4988 tasks      | elapsed:  6.3min
[Parallel(n_jobs=6)]: Done 6038 tasks      | elapsed:  7.6min
[Parallel(n_jobs=6)]: Done 7188 tasks      | elapsed:  9.0min
[Parallel(n_jobs=6)]: Done 7200 out of 7200 | elapsed:  9.0min finished
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y 

GridSearchCV(cv=3, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=6,
             param_grid={'C': [3, 4, 5, 6, 8], 'degree': [2, 3],
                         'epsilon': [0.4, 0.6, 0.8, 1.4, 1.6],
                         'gamma': ['auto', 'scale', 0.1, 1, 3, 10],
                         'max_iter': [500, 1000],
                         'tol': [0.001, 0.01, 0.2, 0.4]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=True)

In [22]:
grid_search.best_estimator_

SVR(C=5, cache_size=200, coef0=0.0, degree=2, epsilon=0.8, gamma='auto',
    kernel='rbf', max_iter=1000, shrinking=True, tol=0.2, verbose=False)

In [23]:
svm_reg_rbf = svm.SVR(kernel='rbf', 
                      C=grid_search.best_estimator_.C, 
                      max_iter=grid_search.best_estimator_.max_iter, 
                      tol=grid_search.best_estimator_.tol, 
                      gamma= grid_search.best_estimator_.gamma,
                      degree = grid_search.best_estimator_.degree )

svm_reg_rbf.fit(X_train, y_train)
preds_rbf = svm_reg_rbf.predict(X_test)

rmse3 = mean_squared_error(y_test, preds_rbf, squared=False)
print("Mean Squared Error: %f" % (rmse3))
max3 = np.sqrt(max_error(y_test, preds_rbf))
print("Max Error: %f" % (max3))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


Mean Squared Error: 2.511703
Max Error: 3.689577


# Polynomial Kernal

Note: will not use, can't get it to work. 

In [ ]:
parameters = {
    'C': [ 1, 2, 3, 4],
    'tol':[0.0005,0.001, 0.003, 0.01],
    'gamma': [ 0.01, 0.05, 0.1, 0.2, 0.4],
    'max_iter': [100, 200, 500, 1000],
    'degree': [2,3,4,5]
}

estimator = svm.SVR(kernel='poly')

In [ ]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    n_jobs = 8,
    cv = 5,
    verbose = True
)

In [ ]:
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 1280 candidates, totalling 6400 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    3.2s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:   30.8s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:  1.2min
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:  2.3min
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:  3.7min
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:  5.4min
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:  7.3min
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:  9.7min
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed: 12.3min
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed: 15.3min
[Parallel(n_jobs=8)]: Done 6034 tasks      | elapsed: 18.5min
[Parallel(n_jobs=8)]: Done 6400 out of 6400 | elapsed: 19.8min finished
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the sha

GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='poly',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=8,
             param_grid={'C': [1, 2, 3, 4], 'degree': [2, 3, 4, 5],
                         'gamma': [0.01, 0.05, 0.1, 0.2, 0.4],
                         'max_iter': [100, 200, 500, 1000],
                         'tol': [0.0005, 0.001, 0.003, 0.01]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=True)

In [ ]:
grid_search.best_estimator_

SVR(C=1, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma=0.01,
    kernel='poly', max_iter=1000, shrinking=True, tol=0.0005, verbose=False)

In [ ]:
svm_reg_poly = svm.SVR(kernel='poly', degree=2, C=1,
                       max_iter=500, tol=0.0005, gamma= 0.01)

svm_reg_poly.fit(X_train, y_train)
preds_poly = svm_reg_poly.predict(X_test)

rmse5 = mean_squared_error(y_test, preds_poly, squared=False)
print("Mean Squared Error: %f" % (rmse5))
max5 = np.sqrt(max_error(y_test, preds_poly))
print("Max Error: %f" % (max5))

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Mean Squared Error: 39983061.174686
Max Error: 9470.494700


/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


# Sigmoid Kernal

In [ ]:
parameters_sig = {
    'C': [ 1, 2, 3, 4, 5, 8, 10, 15],
    'tol':[0.0005,0.001, 0.01],
    'gamma': [ 'auto', 'scale', 0.1, 1, 10],
    'max_iter': [100, 300, 500, 1000],
    'epsilon': [0.25, 0.5, 0.1, 0.15, 0.2],
    'degree':[1,2,3,4]
}

estimator = svm.SVR(kernel='sigmoid')

grid_search_sig = GridSearchCV(
    estimator=estimator,
    param_grid=parameters_sig,
    n_jobs = 8,
    cv = 5,
    verbose = True
)

In [ ]:
grid_search_sig.fit(X_train, y_train)

Fitting 5 folds for each of 9600 candidates, totalling 48000 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  80 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 4016 tasks      | elapsed:    6.8s
[Parallel(n_jobs=8)]: Done 12016 tasks      | elapsed:   18.9s
[Parallel(n_jobs=8)]: Done 23216 tasks      | elapsed:   36.2s
[Parallel(n_jobs=8)]: Done 37616 tasks      | elapsed:   58.9s
[Parallel(n_jobs=8)]: Done 48000 out of 48000 | elapsed:  1.3min finished
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(cv=5, error_score=nan,
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='scale', kernel='sigmoid',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='deprecated', n_jobs=8,
             param_grid={'C': [1, 2, 3, 4, 5, 8, 10, 15],
                         'degree': [1, 2, 3, 4],
                         'epsilon': [0.25, 0.5, 0.1, 0.15, 0.2],
                         'gamma': ['auto', 'scale', 0.1, 1, 10],
                         'max_iter': [100, 300, 500, 1000],
                         'tol': [0.0005, 0.001, 0.01]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=True)

In [ ]:
grid_search.best_estimator_

SVR(C=5, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=50, shrinking=True, tol=0.0005, verbose=False)

In [ ]:
svm_reg_sig = svm.SVR(kernel='sigmoid', 
                      degree=grid_search.best_estimator_.degree, 
                      C=grid_search.best_estimator_.C, 
                      max_iter=grid_search.best_estimator_.max_iter, 
                      tol=grid_search.best_estimator_.tol, 
                      gamma= grid_search.best_estimator_.gamma)

svm_reg_sig.fit(X_train, y_train)
preds_sig = svm_reg_sig.predict(X_test)

rmse4 = mean_squared_error(y_test, preds_sig, squared=False)
print("Mean Squared Error: %f" % (rmse4))
max4 = np.sqrt(max_error(y_test, preds_sig))
print("Max Error: %f" % (max4))

Mean Squared Error: 4.556695
Max Error: 2.283073


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/_base.py:231: ConvergenceWarning: Solver terminated early (max_iter=50).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
